In [88]:
train = read.csv('../data/train.csv', header = TRUE)[,-1] #exclude the ID
test = read.csv('../data/test.csv', header = TRUE)[,-1] #exclude the ID
dim(train)
dim(test)
names(test)[7] = 'Zip'
names(test)

[1] 90  8

[1] 10  7

[1] "YearBuilt"  "SqFt"       "Story"      "Acres"      "Baths"     
[6] "Fireplaces" "Zip"

In [81]:
sum(is.na(train))

[1] 1

In [84]:
#Substitute the NA value to column mean
train$Baths[1] = as.integer(mean(train$Baths[-1]))

In [90]:
#Multiple Linear Regression
#Fit the model on the train set
lm.fit = lm(train$Value~., data = train)
#summary(lm.fit)
#Predict the response on test set
lm.pred = predict(lm.fit, test)
lm.pred

1         2         3         4         5         6         7         8 
232282.97 361120.25  25812.84 395393.18  77930.99 100443.85 153043.40  17656.87 
        9        10 
157812.47 193129.72

In [106]:
#Perform best subset selection on the training data
library(leaps)
Value = train$Value
regfit = regsubsets(Value~., data = train, nvmax = ncol(train)-1)
regfit.summary = summary(best.fit)
#Find the fitted model with the lowest Cp, lowest BIC and the highest adjusted R-squared value
i.cp = which.min(best.summary$cp)
i.bic = which.min(best.summary$bic)
i.adjr2 = which.max(best.summary$adjr2)
coef(regfit, id=i.cp)
coef(regfit, id=i.cp)
coef(regfit, id=i.cp)
#We use YearBuilt, SqFt, Acres, and Zip to predict the property values on the test set
lm.fit = lm(Value~YearBuilt+SqFt+Acres+Zip, data = train)
lm.pred = predict(lm.fit, test)
lm.pred

(Intercept)     YearBuilt          SqFt         Acres           Zip 
-1.163471e+07 -1.723808e+03  1.698484e+02  1.230981e+05  5.406548e+02

(Intercept)     YearBuilt          SqFt         Acres           Zip 
-1.163471e+07 -1.723808e+03  1.698484e+02  1.230981e+05  5.406548e+02

(Intercept)     YearBuilt          SqFt         Acres           Zip 
-1.163471e+07 -1.723808e+03  1.698484e+02  1.230981e+05  5.406548e+02

1         2         3         4         5         6         7         8 
230809.35 341759.67  47664.28 373930.55  67449.51 118907.88 152363.76  28310.43 
        9        10 
139514.65 173977.83